In [ ]:
import sympy as sp
import subprocess
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# configuracion de parametros globales
CONF = {
    "WIDTH": 800,
    "HEIGHT": 800,
    "X_MIN": -2.0,
    "X_MAX": 2.0,
    "Y_MIN": -2.0,
    "Y_MAX": 2.0,
    "MAX_ITER": 50
}

# definicion de la funcion
USER_FUNC_STR = "z**4 + z**3 - 3* z**2 - 1"  
# Otros ejemplos: "z**4 - 1", "z**3 - 2*z + 2"

# crea codigo c necesario
def generate_c_code(func_str):
    print(f"--- ANALIZANDO FUNCIÓN: f(z) = {func_str} ---")
    
    z = sp.symbols('z') # z variable simbolica para sympy
    
    # parsear función, de string a expresion que sympy pueda manejar
    try:
        f_expr = sp.sympify(func_str)
    except:
        print("Error: No se pudo entender la función.")
        sys.exit(1)

    # calcular derivada simbolica automaticamente
    df_expr = sp.diff(f_expr, z)
    print(f"Derivada calculada: f'(z) = {df_expr}")

    # encontrar raices (para pasarlas a C)
    # Convertimos a polinomio para numpy (más robusto numéricamente)
    try:
        poly = sp.Poly(f_expr, z)
        coeffs = poly.all_coeffs() # extrae coeficiente del polinomio
        # calculo de raices, Numpy roots devuelve array de complejos
        roots_np = np.roots(coeffs)
        print(f"Raíces encontradas: {len(roots_np)}")
    except:
        print("La función no es un polinomio simple. El coloreado podría fallar.")
        # Fallback por defecto si no es polinomio
        roots_np = [1+0j, -0.5+0.866j, -0.5-0.866j] 

    # traducir formulas a C
    # Sympy tiene una función 'ccode' que traduce matemáticas a C
    # cpow es la potencia en C, I es imaginario
    f_c = sp.ccode(f_expr).replace('pow', 'cpow') # funcion
    df_c = sp.ccode(df_expr).replace('pow', 'cpow') # derivada

    # generar el array de raices en formato C
    roots_c_str = "{\n"
    for r in roots_np:
        # Escribimos los complejos como a + b * I
        roots_c_str += f"        {r.real} + {r.imag} * I,\n"
    roots_c_str += "    }"

    # PLANTILLA DEL CÓDIGO C
    # Inyectamos f_c, df_c y las raíces en el texto
    c_template = f"""
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <complex.h>
#include <mpi.h>

#define MAX_ITER {CONF['MAX_ITER']}
#define TOL 1e-6
#define TOL_SQUARED (TOL * TOL)
#define NUM_ROOTS {len(roots_np)}

// --- CÓDIGO GENERADO POR PYTHON ---
double complex f(double complex z) {{
    return {f_c};
}}

double complex df(double complex z) {{
    return {df_c};
}}

int main(int argc, char *argv[]) {{

    MPI_Init(&argc, $argv);

    int proceso_raiz;
    MPI_Comm_rank(MPI_COMM_WORLD, &proceso_raiz);

    int num_procesos;
    MPI_Comm_size(MPI_COMM_WORLD, &num_procesos);

    if (argc != 7) {
        if (proceso_raiz == 0) {
             fprintf(stderr, "Uso: %s WIDTH HEIGHT X_MIN X_MAX Y_MIN Y_MAX\n", argv[0]);
        }
        MPI_Finalize();
    
        return 1;
    }

    int width = atoi(argv[1]);
    int height = atoi(argv[2]);
    double x_min = atof(argv[3]);
    double x_max = atof(argv[4]);
    double y_min = atof(argv[5]);
    double y_max = atof(argv[6]);

    // Raices inyectadas desde Python
    const double complex roots[NUM_ROOTS] = {roots_c_str};

    // DIVISIÓN DEL TRABAJO
    // cantidad de filas que calcula cada nucleo
    int fila_por_proceso = height / num_procesos; 
    
    // ultimo proceso tiene las filas que sobran
    if (proceso_raiz == num_procesos - 1) {
        fila_por_proceso = height - proceso_raiz * fila_por_proceso;
    }
    
    // calcula el indice de la fila de inicio para cada proceso
    int y_start = 0; // se usara solo para el calculo
    if (proceso_raiz != 0) {
        y_start = proceso_raiz * (height / num_procesos);
    }
    
    // arreglo en memoria local de cada proceso para almacenar sus resultados
    // cada pixel necesita dos enteros, root_idx e iterations
    // El tipo de dato es int, así que el tamaño es fila_por_proceso * width * sizeof(int) * 2
    int local_data_size = fila_por_proceso * width * 2;
    int *local_data = (int*)malloc(local_data_size * sizeof(int));
    if (!local_data) {
        fprintf(stderr, "Error de asignación de memoria local en rango %d\n", proceso_raiz);
        MPI_Finalize();
        return 1;
    }

    // CÁLCULO DEL FRACTAL (EN PARALELO)
    int data_idx = 0;
    
    // El bucle de filas ahora solo va sobre las filas asignadas a este proceso (y_start a y_start + fila_por_proceso)
    for (int y = y_start; y < y_start + fila_por_proceso; y++) {
        for (int x = 0; x < width; x++) {
            
            // Mapeo (CÓDIGO IDÉNTICO AL SERIAL)
            double re = x_min + (double)x / (width - 1) * (x_max - x_min);
            double im = y_min + (double)y / (height - 1) * (y_max - y_min);
            double complex z = re + im * I;

            int iterations = 0;
            int root_idx = -1;

            for (int k = 0; k < MAX_ITER; k++) {
                double complex deriv = df(z);
                if (cabs(deriv) < 1e-14) break;

                z = z - f(z) / deriv;
                iterations++;

                int converged = 0;
                for (int r = 0; r < NUM_ROOTS; r++) {
                    double complex diff = z - roots[r];
                    if (creal(diff)*creal(diff) + cimag(diff)*cimag(diff) < TOL_SQUARED) {
                        root_idx = r;
                        converged = 1;
                        break;
                    }
                }
                if (converged) break;
            }

            // Guardar resultados en el buffer local
            local_data[data_idx++] = root_idx;
            local_data[data_idx++] = iterations;
        }
    }

    // RECOLECCIÓN DE DATOS (MPI_Gather)
    int *global_data = NULL; // Array completo para el Maestro

    if (proceso_raiz == 0) {
        // Solo el Maestro asigna memoria para todos los datos
        int total_data_size = width * height * 2;
        global_data = (int*)malloc(total_data_size * sizeof(int));
        if (!global_data) {
             fprintf(stderr, "Error de asignación de memoria global\n");
             free(local_data);
             MPI_Finalize();
             return 1;
        }
    }

    // Usamos Gather: todos envían su resultado a global_data en Rank 0
    // Esto es complicado porque el tamaño del último paquete puede variar (residuo)
    // Por simplicidad, usaremos MPI_Gatherv (Vector Gather), que maneja tamaños variables
    
    // Calculamos el tamaño de los bloques (número de enteros a enviar/recibir)
    int *recvcounts = NULL;
    int *displs = NULL;
    int sendcount = local_data_size; // Cada proceso envía su tamaño local

    if (proceso_raiz == 0) {
        // Arrays para MPI_Gatherv para el Maestro
        recvcounts = (int*)malloc(world_size * sizeof(int));
        displs = (int*)malloc(world_size * sizeof(int));
        int displacement = 0;
        
        for (int i = 0; i < world_size; i++) {
            int current_rows = height / world_size;
            if (i == world_size - 1) {
                current_rows = height - i * (height / world_size);
            }
            recvcounts[i] = current_rows * width * 2; // Número de enteros recibidos
            displs[i] = displacement;                // Dónde empieza a guardar este bloque
            displacement += recvcounts[i];
        }
    }
    
    // La recolección de datos (Gatherv)
    MPI_Gatherv(
        local_data, // Buffer de envío
        sendcount, // Tamaño del envío local
        MPI_INT,
        global_data, // Buffer de recepción (solo significativo en Rank 0)
        recvcounts, // Tamaños de recepción (solo significativo en Rank 0)
        displs, // Desplazamientos (solo significativo en Rank 0)
        MPI_INT,
        0, // Rank de destino (Maestro)
        MPI_COMM_WORLD
    );

    // ESCRITURA DE ARCHIVO (SOLO EN EL MAESTRO)
    if (proceso_raiz == 0) {
        FILE *fp = fopen("fractal_data.csv", "w");
        if (!fp) {
            fprintf(stderr, "Error al crear CSV.\n");
            free(global_data);
            free(recvcounts);
            free(displs);
            MPI_Finalize();
            return 1;
        }
        
        // Escribimos la cabecera
        fprintf(fp, "root_idx,iterations\n");
        
        // Escribimos los datos globales
        for (int i = 0; i < height * width * 2; i += 2) {
            fprintf(fp, "%d,%d\n", global_data[i], global_data[i+1]);
        }
        
        fclose(fp);
        free(global_data);
        free(recvcounts);
        free(displs);
        printf("Fractal generado y guardado por el Maestro.\n");
    }

    // FINALIZACIÓN
    free(local_data);
    MPI_Finalize(); // Siempre llamar a MPI_Finalize
    return 0;
}}
"""
    # escribe el codigo C en el archivo
    with open("fractal_generated.c", "w") as f:
        f.write(c_template)
    print("Archivo 'fractal_generated.c' creado dinámicamente.")

# compilador y ejecucion del calculo
def compile_and_run():
    # compilar
    exe_name = "fractal_gen"
    if os.name == 'nt': exe_name += ".exe"
    
    cmd_compile = ["gcc", "fractal_generated.c", "-o", exe_name, "-lm", "-O3"]
    subprocess.run(cmd_compile, check=True)
    
    # ejecutar
    cmd_run = [
        "./" + exe_name if os.name != 'nt' else exe_name,
        str(CONF["WIDTH"]), str(CONF["HEIGHT"]),
        str(CONF["X_MIN"]), str(CONF["X_MAX"]),
        str(CONF["Y_MIN"]), str(CONF["Y_MAX"])
    ]
    subprocess.run(cmd_run, check=True) # aqui ocurre el calculo del fractal

def plot_fractal():
    print("--- GRAFICANDO ---")
    df = pd.read_csv("fractal_data.csv")
    w, h = CONF["WIDTH"], CONF["HEIGHT"]
    
    roots = df['root_idx'].values.reshape((h, w)) # reestructura lo datos del indice en una matrix height x width
    iters = df['iterations'].values.reshape((h, w)) # reestructura los datos de iteraciones en una amtrix height x width 
    
    # paleta de colores dinamica segun cantidad de raices
    num_roots = roots.max() + 1
    # Usamos un mapa de colores de matplotlib 
    cmap = plt.get_cmap('tab10', num_roots) 
    
    # crear imagen RGB
    img = np.zeros((h, w, 3))
    
    # asignar colores base
    mask_conv = roots >= 0
    for r in range(num_roots):
        r_mask = (roots == r)
        color = cmap(r)[:3] # Tomar RGB del mapa
        img[r_mask] = color
        
    # Aplicar sombras por iteraciones
    brightness = 1.0 - (iters / CONF["MAX_ITER"]) * 0.6
    for c in range(3): img[..., c] *= brightness
    img[~mask_conv] = 0.0

    plt.figure(figsize=(8,8))
    plt.imshow(img, extent=[CONF["X_MIN"], CONF["X_MAX"], CONF["Y_MIN"], CONF["Y_MAX"]], origin='lower')
    plt.title(f"Fractal de Newton: f(z) = {USER_FUNC_STR}")
    plt.xlabel("Re(z)")
    plt.ylabel("Im(z)")
    plt.show()

if __name__ == "__main__":
    # 1. Python calcula derivadas y raíces, y escribe el C
    generate_c_code(USER_FUNC_STR)
    
    # 2. Compilar y correr el C generado
    compile_and_run()
    
    # 3. Mostrar resultado
    plot_fractal()